# **Import**

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error

# **Data Load**

In [3]:
cd /content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Data

/content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Data


In [4]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = [f's_{i + 1}' for i in range(21)]
col_names = index_names + setting_names + sensor_names

train_df = pd.read_csv('./train_FD004.csv')

In [5]:
max_cycle = train_df.groupby('unit_number')['time_cycles'].max().reset_index()
max_cycle.columns = ['unit_number', 'max_cycle']

train_df = train_df.merge(max_cycle, on='unit_number', how='left')

train_df['RUL'] = train_df['max_cycle'] - train_df['time_cycles']

In [6]:
test_df = pd.read_csv('./test_FD004.csv')
rul_df = pd.read_csv('./RUL_FD004.csv')

In [7]:
results = []

using_sensors = [
    's_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12',
    's_13', 's_14', 's_15', 's_17', 's_20', 's_21'
]

In [8]:
train_df['setting_1'] = train_df['setting_1'].round(1)
train_df['setting_2'] = train_df['setting_2'].round(1)
train_df['setting_group'] = train_df[['setting_1','setting_2','setting_3']].astype(str).agg('_'.join, axis=1)
train_df = pd.get_dummies(train_df, columns=['setting_group'])
train_df.head()

,Unnamed: 0,unit_number,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,...,s_21,max_cycle,RUL,setting_group_0.0_0.0_100.0,setting_group_10.0_0.2_100.0,setting_group_10.0_0.3_100.0,setting_group_20.0_0.7_100.0,setting_group_25.0_0.6_60.0,setting_group_35.0_0.8_100.0,setting_group_42.0_0.8_100.0
0,0,1,1,42.0,0.8,100.0,445.00,549.68,1343.43,1112.93,...,6.3670,321,320,False,False,False,False,False,False,True
1,1,1,2,20.0,0.7,100.0,491.19,606.07,1477.61,1237.50,...,14.6552,321,319,False,False,False,True,False,False,False
2,2,1,3,42.0,0.8,100.0,445.00,548.95,1343.12,1117.05,...,6.4213,321,318,False,False,False,False,False,False,True
3,3,1,4,42.0,0.8,100.0,445.00,548.70,1341.24,1118.03,...,6.4176,321,317,False,False,False,False,False,False,True
4,4,1,5,25.0,0.6,60.0,462.54,536.10,1255.23,1033.59,...,8.6754,321,316,False,False,False,False,True,False,False


In [9]:
setting_group = [col for col in train_df.columns if col.startswith('setting_group')]
setting_group

['setting_group_0.0_0.0_100.0',
 'setting_group_10.0_0.2_100.0',
 'setting_group_10.0_0.3_100.0',
 'setting_group_20.0_0.7_100.0',
 'setting_group_25.0_0.6_60.0',
 'setting_group_35.0_0.8_100.0',
 'setting_group_42.0_0.8_100.0']

In [10]:
test_df['setting_1'] = test_df['setting_1'].round(1)
test_df['setting_2'] = test_df['setting_2'].round(1)
test_df['setting_group'] = test_df[['setting_1','setting_2','setting_3']].astype(str).agg('_'.join, axis=1)
test_df = pd.get_dummies(test_df, columns=['setting_group'])
test_df.head()

,Unnamed: 0,unit_number,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,...,s_19,s_20,s_21,setting_group_0.0_0.0_100.0,setting_group_10.0_0.2_100.0,setting_group_10.0_0.3_100.0,setting_group_20.0_0.7_100.0,setting_group_25.0_0.6_60.0,setting_group_35.0_0.8_100.0,setting_group_42.0_0.8_100.0
0,0,1,1,20.0,0.7,100.0,491.19,606.67,1481.04,1227.81,...,100.00,24.31,14.7007,False,False,False,True,False,False,False
1,1,1,2,25.0,0.6,60.0,462.54,536.22,1256.17,1031.48,...,84.93,14.36,8.5748,False,False,False,False,True,False,False
2,2,1,3,42.0,0.8,100.0,445.00,549.23,1340.13,1105.88,...,100.00,10.39,6.4365,False,False,False,False,False,False,True
3,3,1,4,42.0,0.8,100.0,445.00,549.19,1339.70,1107.26,...,100.00,10.56,6.2367,False,False,False,False,False,False,True
4,4,1,5,35.0,0.8,100.0,449.44,555.10,1353.04,1117.80,...,100.00,14.85,8.9326,False,False,False,False,False,True,False


## **Raw Data**

In [11]:
# RUL Clipping X
raw_results = []

dt = DecisionTreeRegressor(random_state=42)

model_dict = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'Bagging': BaggingRegressor(estimator=dt, random_state=42)
}

for model_name, model in model_dict.items():
    train_x_sensors = train_df[using_sensors].copy()
    train_x_settings = train_df[setting_group].copy()
    train_y = train_df['RUL'].copy().values.ravel()
    test_x_sensors = test_df.groupby('unit_number').tail(1)[using_sensors]
    test_x_settings = test_df.groupby('unit_number').tail(1)[setting_group]

    train_x = np.concatenate([train_x_sensors, train_x_settings], axis=1)
    test_x = np.concatenate([test_x_sensors, test_x_settings], axis=1)

    model.fit(train_x, train_y)

    pred = model.predict(test_x)
    true = rul_df['RUL'].copy().values.ravel()

    mae = mean_absolute_error(true, pred)
    rmse = mean_squared_error(true, pred)**0.5

    print(f'Model: {model_name}, MAE: {mae}, RMSE: {rmse}')

    raw_results.append({
    'Data': 'Raw',
    'Model': model_name,
    'Scaler': 'None',
    'MAE': mae,
    'RMSE': rmse
    })
results.extend(raw_results)

Model: RandomForest, MAE: 32.99782258064516, RMSE: 45.37433864265202
Model: Bagging, MAE: 34.63467741935484, RMSE: 48.27586136282377


In [12]:
pd.DataFrame(raw_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
0,Raw,RandomForest,None,32.997823,45.374339
1,Raw,Bagging,None,34.634677,48.275861


In [13]:
# RUL Clipping O
raw_clipping_results = []

dt = DecisionTreeRegressor(random_state=42)

model_dict = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'Bagging': BaggingRegressor(estimator=dt, random_state=42)
}

for model_name, model in model_dict.items():
    train_x_sensors = train_df[using_sensors].copy()
    train_x_settings = train_df[setting_group].copy()
    train_y = train_df['RUL']
    test_x_sensors = test_df.groupby('unit_number').tail(1)[using_sensors]
    test_x_settings = test_df.groupby('unit_number').tail(1)[setting_group]

    train_y.loc[train_y > 125] = 125
    train_y = train_y.values.ravel()

    train_x = np.concatenate([train_x_sensors, train_x_settings], axis=1)
    test_x = np.concatenate([test_x_sensors, test_x_settings], axis=1)

    model.fit(train_x, train_y)

    pred = model.predict(test_x)
    true = rul_df['RUL'].copy()
    true.loc[true > 125] = 125
    true = true.values.ravel()

    mae = mean_absolute_error(true, pred)
    rmse = mean_squared_error(true, pred)**0.5

    print(f'Model: {model_name}, MAE: {mae}, RMSE: {rmse}')

    raw_clipping_results.append({
    'Data': 'Raw (RUL Clipping)',
    'Model': model_name,
    'Scaler': 'None',
    'MAE': mae,
    'RMSE': rmse
    })
results.extend(raw_clipping_results)

Model: RandomForest, MAE: 14.622258064516128, RMSE: 19.808459764095904
Model: Bagging, MAE: 14.537096774193548, RMSE: 20.125729800432083


In [14]:
pd.DataFrame(raw_clipping_results).sort_values(by='MAE')

,Data,Model,Scaler,MAE,RMSE
1,Raw (RUL Clipping),Bagging,None,14.537097,20.12573
0,Raw (RUL Clipping),RandomForest,None,14.622258,19.80846


## **Cumulative Stat**

In [15]:
cum_stat_train_df = pd.read_csv('./cum_stat_train_FD004.csv')
cum_stat_test_df = pd.read_csv('./cum_stat_test_FD004.csv')

In [16]:
stat_cols = [col for col in cum_stat_train_df.columns if col.startswith('s_')]

In [17]:
# RUL Clipping X
cum_stat_results = []

dt = DecisionTreeRegressor(random_state=42)

model_dict = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'Bagging': BaggingRegressor(estimator=dt, random_state=42)
}

for model_name, model in model_dict.items():
    train_x_sensors = cum_stat_train_df[stat_cols].copy()
    train_x_settings = cum_stat_train_df[setting_group].copy()
    train_y = cum_stat_train_df['RUL'].copy().values.ravel()
    test_x_sensors = cum_stat_test_df.groupby('unit_number').tail(1)[stat_cols]
    test_x_settings = cum_stat_test_df.groupby('unit_number').tail(1)[setting_group]

    train_x = np.concatenate([train_x_sensors, train_x_settings], axis=1)
    test_x = np.concatenate([test_x_sensors, test_x_settings], axis=1)

    model.fit(train_x, train_y)

    pred = model.predict(test_x)
    true = rul_df['RUL'].copy().values.ravel()

    mae = mean_absolute_error(true, pred)
    rmse = mean_squared_error(true, pred)**0.5

    print(f'Model: {model_name}, MAE: {mae}, RMSE: {rmse}')

    cum_stat_results.append({
    'Data': 'Cumulative Stat',
    'Model': model_name,
    'Scaler': 'None',
    'MAE': mae,
    'RMSE': rmse
    })
results.extend(cum_stat_results)

Model: RandomForest, MAE: 22.77923387096774, RMSE: 31.577346750836757
Model: Bagging, MAE: 23.209274193548385, RMSE: 32.62798805992655


In [18]:
pd.DataFrame(cum_stat_results)

,Data,Model,Scaler,MAE,RMSE
0,Cumulative Stat,RandomForest,None,22.779234,31.577347
1,Cumulative Stat,Bagging,None,23.209274,32.627988


In [19]:
# RUL Clipping O
cum_stat_rul_clipping_results = []

dt = DecisionTreeRegressor(random_state=42)

model_dict = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'Bagging': BaggingRegressor(estimator=dt, random_state=42)
}

for model_name, model in model_dict.items():
    train_x_sensors = cum_stat_train_df[stat_cols].copy()
    train_x_settings = cum_stat_train_df[setting_group].copy()
    train_y = cum_stat_train_df['RUL'].copy()
    test_x_sensors = cum_stat_test_df.groupby('unit_number').tail(1)[stat_cols]
    test_x_settings = cum_stat_test_df.groupby('unit_number').tail(1)[setting_group]

    train_y.loc[train_y > 125] = 125
    train_y = train_y.values.ravel()

    train_x = np.concatenate([train_x_sensors, train_x_settings], axis=1)
    test_x = np.concatenate([test_x_sensors, test_x_settings], axis=1)

    model.fit(train_x, train_y)

    pred = model.predict(test_x)
    true = rul_df['RUL'].copy()
    true.loc[true > 125] = 125
    true = true.values.ravel()

    mae = mean_absolute_error(true, pred)
    rmse = mean_squared_error(true, pred)**0.5

    print(f'Model: {model_name}, MAE: {mae}, RMSE: {rmse}')

    cum_stat_rul_clipping_results.append({
    'Data': 'Cumulative Stat (RUL Clipping)',
    'Model': model_name,
    'Scaler': 'None',
    'MAE': mae,
    'RMSE': rmse
    })
results.extend(cum_stat_rul_clipping_results)

Model: RandomForest, MAE: 14.085685483870968, RMSE: 20.226422883478193
Model: Bagging, MAE: 14.515725806451613, RMSE: 21.49189926160477


In [20]:
pd.DataFrame(cum_stat_rul_clipping_results)

,Data,Model,Scaler,MAE,RMSE
0,Cumulative Stat (RUL Clipping),RandomForest,None,14.085685,20.226423
1,Cumulative Stat (RUL Clipping),Bagging,None,14.515726,21.491899


# **Result**

In [21]:
result = pd.DataFrame(results)
result.to_csv('/content/drive/MyDrive/[Projects]/Kaggle/NASA 터보팬 제트 엔진의 잔존 수명 예측 및 예지보전 방안/Results_FD004/Bagging_Regression.csv')

In [22]:
result = result.sort_values(by='MAE')
result.head()

,Data,Model,Scaler,MAE,RMSE
6,Cumulative Stat (RUL Clipping),RandomForest,None,14.085685,20.226423
7,Cumulative Stat (RUL Clipping),Bagging,None,14.515726,21.491899
3,Raw (RUL Clipping),Bagging,None,14.537097,20.125730
2,Raw (RUL Clipping),RandomForest,None,14.622258,19.808460
4,Cumulative Stat,RandomForest,None,22.779234,31.577347
